# Assignment 3: Question 3
#### By Hunter Gallant

---
## Pre-processing

In [1]:
f = open("A25_HW3_stardata.dat")
raw_data = f.readlines()
f.close()

In [2]:
# Processing the raw input data
# Table columns: | Type | M_V | BC | L (L_sol) | B-V |
stardata = []
for line in raw_data[1:]:
    raw_row = line.split(" ")
    row = []
    for cell in raw_row:
        if len(cell) != 0:
            row.append(cell)
        
    clean_row = []
    for i in range(len(row)):
        if i == 0: val = row[i]
        elif 0 < i < 4: val = float(row[i])
        else: val = float(row[i][:-1])
        clean_row.append(val)
    stardata.append(clean_row)
#print(stardata)

---
## Part A

$$1M_{\odot}=C\int_{0.2}^{50}m^{-1.35}dm$$

In [3]:
def makeBins(delta_m, lo, hi):
    bins = [lo]
    num_bins = int(hi/delta_m) - int(lo/delta_m)
    for i in range(num_bins):
        bins.append(round(bins[-1]+delta_m, 5))
    return bins

In [4]:
def partA(delta_m):
    bins = makeBins(delta_m, 0.2, 50)
    sum_a = 0
    for b in bins:
        sum_a += (b**(-1.35))*delta_m
    print("Sum: " + str(sum_a))
    C = 1/sum_a
    print("C: " + str(C))

In [5]:
partA(0.001)

Sum: 4.296275006090222
C: 0.2327597741258279


---
## Part B

$$L_{\textrm{bol}}= 2.328*10^9 \int_{0.2}^{50}L(m)m^{-2.35}dm$$

In [6]:
def partB(delta_m):
    bins = makeBins(delta_m, 0.2, 50)
    sum_b = 0
    for m in bins:
        # L(m) changes depending on the value of m
        if m < 0.5:
            sum_b += (0.27 * m**(2.6) * m**(-2.35)) * delta_m
        elif m < 2:
            sum_b += (m**(4.5) * m**(-2.35)) * delta_m
        elif m < 20:
            sum_b += (1.9 * m**(3.6) * m**(-2.35)) * delta_m
        else:
            sum_b += (4600 * m * m**(-2.35)) * delta_m
    front = 2.328 * 10**9
    total = front * sum_b
    return total

In [7]:
partB(0.001)/10**12

4.6021571806261345

---
## Part D

$$L_V = 2.328*10^9\int_{0.2}^{50}L(m)*10^{0.4BC}*m^{-2.35}dm$$
$$L_B = 2.328*10^9\int_{0.2}^{50}L(m)*10^{0.4|BC-(B-V)|}*m^{-2.35}dm$$

In [18]:
from scipy.interpolate import interp1d
import numpy as np

In [97]:
# Table columns: | Type | M_V | BC | L (L_sol) | B-V |
def interpolation():
    # Use BC to L relationship to interpolate for all values of L
    
    bc_values = []
    l_values = []
    bv_values = []
    
    for row in stardata:
        bc_values.append(row[2])
        l_values.append(row[3])
        bv_values.append(row[4])
    
    # SciPy interpolation --> interp1d
    # https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html
    interpolate_bc = interp1d(l_values, bc_values, fill_value="extrapolate")
    interpolate_bv = interp1d(l_values, bv_values, fill_value="extrapolate")
    
    # return the interpolation functions so this function only needs to get called once
    return interpolate_bc, interpolate_bv

In [102]:
def partD(delta_m):
    bins = makeBins(delta_m, 0.2, 50)
    # sums of V and B bands
    sum_v = 0
    sum_b = 0
    
    # only call interpolation once
    int_bc, int_bv = interpolation()
    
    for m in bins:
        # L(m) changes depending on the value of m
        if m < 0.5:
            L = 0.27 * m**(2.6)
        elif m < 2:
            L = m**(4.5)
        elif m < 20:
            L = 1.9 * m**(3.6)
        else:
            L = 4600 * m
        # BC and (B-V) interpolation
        temp_bc = int_bc([L])
        temp_bv = int_bv([L])
        
        sum_v += L * 10**(0.4 * temp_bc) * m**(-2.35) * delta_m
        sum_b += L * 10**(0.4 * abs(temp_bc-temp_bv)) * m**(-2.35) * delta_m 
    
    front = 2.328 * 10**9
    total_v = front * sum_v
    total_b = front * sum_b
    
    return total_v, total_b

In [104]:
total_v, total_b = partD(0.001)
print(total_v)
print(total_b)

[3.36246889e+11]
[6.69756081e+13]
